In [3]:
from bs4 import BeautifulSoup
import urllib
import requests
import selenium
import pandas as pd
import os

In [7]:
# Load job and location function
def load_indeed_jobs_div(job_title, location):
    getVars = {'q' : job_title, 'l' : location, 'fromage': 'last', 'sort' : 'date'}
    url = ('https://www.indeed.co.uk/jobs?' + urllib.parse.urlencode(getVars))
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    job_soup = soup.find(id="resultsCol")
    return job_soup

job_soup = load_indeed_jobs_div('data scientist', 'London')

In [38]:
# Extract job title
def extract_job_title_indeed(job_elem):
    title_elem = job_elem.find('h2', class_='title')
    title = title_elem.text.strip()
    return title

# Extract company
def extract_company_indeed(job_elem):
    company_elem = job_elem.find('span', class_='company')
    company = company_elem.text.strip()
    return company

# Extract salary
def extract_salary_indeed(job_elem):
    salary_elem = job_elem.find('span', attrs={'class': 'salaryText'})
    if salary_elem:
        salary = salary_elem.string.strip()
    else:
        salary = 'Not shown'
    return salary

# Extract date
def extract_date_indeed(job_elem):
    date_elem = job_elem.find('span', class_='date')
    date = date_elem.text.strip()
    return date


In [57]:
def extract_job_information_indeed(job_soup):
    job_elems = job_soup.find_all('div', class_='jobsearch-SerpJobCard')
     
    extracted_info = []
    dataframe = pd.DataFrame(columns=["Title", "Company", "Salary"])
    
    titles = [extract_job_title_indeed(job_elem) for job_elem in job_elems]
    extracted_info.append(titles)       
    
    salary = [extract_salary_indeed(job_elem) for job_elem in job_elems]
    extracted_info.append(salary)    
    
    companies = [extract_company_indeed(job_elem) for job_elem in job_elems]
    extracted_info.append(companies)     

    date = [extract_date_indeed(job_elem) for job_elem in job_elems]
    extracted_info.append(date)     
    
    dataframe = pd.DataFrame({'Title': titles,
                                  "Company": companies,
                                  "Salary": salary})
    
    return dataframe

In [58]:
jobs_df = extract_job_information_indeed(job_soup = job_soup)
jobs_df

,Title,Company,Salary
0,Data Scientist\nnew,Williams Lea,Not shown
1,Data Science Associate - FS Innovation Hub - L...,Ernst & Young,Not shown
2,Principal Data Scientist - Datalab\nnew,BBC,Not shown
3,Data Scientist\nnew,ECS Recruitment Group,£450 a week
4,Data Science Senior - FS Innovation Hub - Lond...,Ernst & Young,Not shown
5,"Software Engineer (Machine Learning, AI, Data ...",Ernest Gordon Recruitment Limited,"£30,000 - £70,000 a year"
6,Group Director of Data & Analytics\nnew,The Specialist Works,Not shown
7,Sr. Data Scientist\nnew,Reputation,Not shown
8,Program Manager (Data Science Projects) - Asso...,"JPMorgan Chase Bank, N.A.",Not shown
9,Senior P&C Analytics Solutions Manager\nnew,Swiss Re,Not shown


In [ ]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import time

driver = webdriver.Firefox(executable_path="c:/program/geckodriver.exe")
driver.maximize_window()
dataframe = pd.DataFrame(columns=["Title", "Location", "Company", "Salary", "Description"])

for i in range(0, 10, 10):

    driver.get("https://www.indeed.com/jobs?q=Senior+Software+Engineer&l=Philadelphia%2C+PA&start=" + str(i))
    driver.implicitly_wait(5)

    all_jobs = driver.find_elements_by_class_name('result')

    for job in all_jobs:

        result_html = job.get_attribute('innerHTML')
        soup = BeautifulSoup(result_html, 'html.parser')

        try:
            title = soup.find("a", class_="jobtitle").text.replace('\n', '')
        except:
            title = 'None'

        try:
            location = soup.find(class_="location").text
        except:
            location = 'None'

        try:
            company = soup.find(class_="company").text.replace("\n", "").strip()
        except:
            company = 'None'

        try:
            salary = soup.find(class_="salary").text.replace("\n", "").strip()
        except:
            salary = 'None'

        sum_div = job.find_elements_by_class_name("summary")[0]
        try:
            sum_div.click()
        except:
            close_button = driver.find_elements_by_class_name("popover-x-button-close")[0]
            close_button.click()
            sum_div.click()
        try:
            jd = driver.find_element_by_css_selector('div#vjs-desc').text
            #print(jd)
        except:
            jd = 'None'

        dataframe = dataframe.append({'Title': title,
                                      'Location': location,
                                      "Company": company,
                                      "Salary": salary,
                                      "Description": jd},
                                     ignore_index=True)

dataframe.to_csv("c.csv", index=False)